# Task 2 — TSLA Time Series Forecasting (ARIMA/SARIMA vs Multivariate LSTM)

This notebook is the **only** notebook for Task 2. **All modeling and artifact generation is done via scripts** (see `scripts/04_task2_train_arima.py`, `scripts/05_task2_train_lstm.py`, `scripts/06_task2_compare_models.py`).

## Required deliverables shown here
1. **Chronological split** with cutoff on the **last trading day of 2024** (loaded from `split_info.json`).
2. **ARIMA/SARIMA model specification** (loaded from `arima_params.json`).
3. **LSTM architecture + training configuration** (loaded from `lstm_architecture.json`).
4. **Forecast CSVs aligned to test dates** (`tsla_arima_forecast.csv`, `tsla_lstm_forecast.csv`, `tsla_forecasts_merged.csv`).
5. **Performance metrics table** with MAE/RMSE/MAPE (`model_comparison.csv`).
6. A brief **discussion**: which model performed better and why, grounded in the metrics and behavior.

---

## Additional (demonstration) workflow sections
To make the end-to-end methodology transparent, this notebook also **demonstrates** the standard time series workflow steps using the saved split/feature datasets (created by scripts):

### A) Classical time series forecasting workflow (ARIMA/SARIMA)
1. Load TSLA time series
2. Explore and understand the data
3. Test stationarity (ADF)
4. **Demo:** build an ARIMA model on *returns* (stationary)
5. Forecast prices (via returns → price path)
6. Visualize and interpret results

### B) Deep learning workflow (LSTM)
1. Load TSLA time series
2. Explore and understand the data
3. Test stationarity (ADF)
4. **Demo:** prepare data for LSTM (scaling + sequence creation)
5. (Training is done in scripts)
6. Forecast prices (loaded from artifacts)
7. Visualize and interpret results
8. Evaluate forecast accuracy

> Important: the **official submission outputs** remain the script-generated artifacts in `outputs/task2/*`.


In [ ]:
# Standard imports
import os
import json
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 120)

def _find_repo_root(start: Path) -> Path:
    """Walk upward until we find a folder containing both `src/` and `outputs/`."""
    start = start.resolve()
    for candidate in [start, *start.parents]:
        if (candidate / 'src').is_dir() and (candidate / 'outputs').exists():
            return candidate
    return start

# Ensure REPO_ROOT is always a concrete Path
_existing_repo_root = globals().get('REPO_ROOT', None)
if isinstance(_existing_repo_root, Path):
    REPO_ROOT = _existing_repo_root
else:
    REPO_ROOT = _find_repo_root(Path.cwd())
    globals()['REPO_ROOT'] = REPO_ROOT
    print('Repo root (auto-detected):', REPO_ROOT)

print('Notebook working directory:', os.getcwd())
print('Repo root:', REPO_ROOT)


## 0) Define paths (repo-relative)

These paths match the script outputs defined in `src/config.py`.


In [ ]:
# --- Script-generated artifacts (required deliverables) ---
SPLIT_INFO_PATH = REPO_ROOT / 'outputs' / 'task2' / 'metrics' / 'split_info.json'
ARIMA_PARAMS_PATH = REPO_ROOT / 'outputs' / 'task2' / 'metrics' / 'arima_params.json'
LSTM_ARCH_PATH = REPO_ROOT / 'outputs' / 'task2' / 'metrics' / 'lstm_architecture.json'
MODEL_COMPARISON_PATH = REPO_ROOT / 'outputs' / 'task2' / 'metrics' / 'model_comparison.csv'
ERROR_DIAGNOSTICS_PATH = REPO_ROOT / 'outputs' / 'task2' / 'metrics' / 'error_diagnostics.csv'

ARIMA_FORECAST_PATH = REPO_ROOT / 'outputs' / 'task2' / 'forecasts' / 'tsla_arima_forecast.csv'
LSTM_FORECAST_PATH = REPO_ROOT / 'outputs' / 'task2' / 'forecasts' / 'tsla_lstm_forecast.csv'
MERGED_FORECASTS_PATH = REPO_ROOT / 'outputs' / 'task2' / 'forecasts' / 'tsla_forecasts_merged.csv'

FORECAST_PLOT_PATH = REPO_ROOT / 'outputs' / 'task2' / 'figures' / 'forecast_test_period.png'

required_files = [
    SPLIT_INFO_PATH,
    ARIMA_PARAMS_PATH,
    LSTM_ARCH_PATH,
    MODEL_COMPARISON_PATH,
    ERROR_DIAGNOSTICS_PATH,
    ARIMA_FORECAST_PATH,
    LSTM_FORECAST_PATH,
    MERGED_FORECASTS_PATH,
]

missing = [p for p in required_files if not p.exists()]
if missing:
    print('Missing required artifacts. Run scripts first:')
    for m in missing:
        print(' -', m)
else:
    print('All required artifacts exist.')

print('Optional plot exists:', FORECAST_PLOT_PATH.exists())

# --- (NEW) Underlying split/feature datasets used for workflow demos ---
TRAIN_SPLIT_PATH = REPO_ROOT / 'data' / 'task2' / 'splits' / 'tsla_train.parquet'
TEST_SPLIT_PATH  = REPO_ROOT / 'data' / 'task2' / 'splits' / 'tsla_test.parquet'

FEATURES_TRAIN_PATH = REPO_ROOT / 'data' / 'task2' / 'features' / 'tsla_features_train.parquet'
FEATURES_TEST_PATH  = REPO_ROOT / 'data' / 'task2' / 'features' / 'tsla_features_test.parquet'

optional_inputs = [TRAIN_SPLIT_PATH, TEST_SPLIT_PATH, FEATURES_TRAIN_PATH, FEATURES_TEST_PATH]
missing_inputs = [p for p in optional_inputs if not p.exists()]
print('Underlying split/feature files present:', len(missing_inputs) == 0)
if missing_inputs:
    print('Some optional inputs are missing (demo sections may be skipped):')
    for p in missing_inputs:
        print(' -', p)


## Time Series Forecasting: ARIMA vs SARIMA vs LSTM (context)

### Goal
Forecast future values using patterns in past observations.

### Models Compared
- **ARIMA/SARIMA (Classical):** linear, interpretable, strong baselines
- **LSTM (Modern):** non-linear, can learn complex temporal patterns and interactions among multiple features

### Key Decision Factors
- Seasonality present? (→ consider SARIMA)
- Data size (small → classical often strong; larger + richer features → deep learning may help)
- Interpretability needs

### Workflow
`Clean → Check stationarity/seasonality → Model → Evaluate`

> In this project, the **official trained models and forecasts** are generated by scripts and loaded below as artifacts.


## 1) Load split information (proof of correct chronological cutoff)

Task requirement: chronological split where training ends on the **last trading day of 2024**.


In [ ]:
with SPLIT_INFO_PATH.open('r', encoding='utf-8') as f:
    split_info = json.load(f)

display(split_info)

split_summary = pd.DataFrame([split_info])
display(split_summary)


## (Demo) Load TSLA series + explore

This section demonstrates the typical first steps: load a single ticker time series, check coverage and missingness, and visualize prices/returns.


In [ ]:
if TRAIN_SPLIT_PATH.exists() and TEST_SPLIT_PATH.exists():
    tsla_train = pd.read_parquet(TRAIN_SPLIT_PATH).copy()
    tsla_test  = pd.read_parquet(TEST_SPLIT_PATH).copy()

    for df in (tsla_train, tsla_test):
        df['date'] = pd.to_datetime(df['date'])
        df.sort_values('date', inplace=True)

    tsla_full = (
        pd.concat([tsla_train, tsla_test], ignore_index=True)
          .sort_values('date')
          .drop_duplicates(subset=['date'])
          .reset_index(drop=True)
    )

    print('Train:', tsla_train.shape, tsla_train['date'].min(), '→', tsla_train['date'].max())
    print('Test: ', tsla_test.shape,  tsla_test['date'].min(),  '→', tsla_test['date'].max())
    print('Full: ', tsla_full.shape,  tsla_full['date'].min(),  '→', tsla_full['date'].max())

    display(tsla_full.head())
    display(tsla_full.tail())
    print('Missing values:\n', tsla_full.isna().sum())

    plt.figure(figsize=(12, 4))
    plt.plot(tsla_full['date'], tsla_full['adj_close'], linewidth=1.5)
    plt.title('TSLA adjusted close (train + test)')
    plt.xlabel('Date')
    plt.ylabel('Adj Close')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # log returns
    tsla_full['log_ret'] = np.log(tsla_full['adj_close']).diff()

    plt.figure(figsize=(12, 3))
    plt.plot(tsla_full['date'], tsla_full['log_ret'], linewidth=1)
    plt.title('TSLA log returns')
    plt.xlabel('Date')
    plt.ylabel('log return')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    display(tsla_full['log_ret'].describe())
else:
    print('Skipping underlying-series EDA: train/test parquet files not found.')


## (Demo) Test stationarity using the ADF test

Typically, price levels are non-stationary, while returns are closer to stationary.


In [ ]:
if 'tsla_full' in globals():
    from typing import Mapping, Tuple, cast

    from statsmodels.tsa.stattools import adfuller

    def adf_report(series, name):
        series = pd.Series(series).dropna()

        # `adfuller` has multiple return shapes depending on args.
        # We always call it with autolag='AIC' (=> includes `icbest`) and default `store=False`.
        ADFResult = Tuple[float, float, int, int, Mapping[str, float], float]
        result = cast(ADFResult, adfuller(series, autolag='AIC'))

        stat, pvalue, usedlag, nobs, crit, icbest = result

        print(f'ADF test — {name}')
        print(f'  test statistic: {stat:.4f}')
        print(f'  p-value:        {pvalue:.6f}')
        print(f'  lags used:      {usedlag}')
        print(f'  nobs:           {nobs}')
        print('  critical values:', {k: float(v) for k, v in crit.items()})
        print(f'  icbest:         {icbest:.4f}')
        print('  conclusion:', 'stationary (reject unit root)' if pvalue < 0.05 else 'non-stationary (fail to reject)')
        print()

    adf_report(tsla_full['adj_close'], 'adj_close (price level)')
    adf_report(tsla_full['log_ret'], 'log returns')
else:
    print('Skipping ADF test: underlying series not loaded.')


## 2) Load forecasts (aligned to test dates)

We load:
- ARIMA forecast CSV
- LSTM forecast CSV
- Merged forecast CSV (should align by test dates)


In [ ]:
arima_fc = pd.read_csv(ARIMA_FORECAST_PATH)
lstm_fc = pd.read_csv(LSTM_FORECAST_PATH)
merged_fc = pd.read_csv(MERGED_FORECASTS_PATH)

for df in (arima_fc, lstm_fc, merged_fc):
    df['date'] = pd.to_datetime(df['date'])

print('ARIMA forecast shape:', arima_fc.shape)
print('LSTM forecast shape:', lstm_fc.shape)
print('Merged forecast shape:', merged_fc.shape)

display(arima_fc.head())
display(lstm_fc.head())
display(merged_fc.head())


### Forecast alignment checks
We verify that:
- test dates are sorted
- merged dates match ARIMA dates
- merged contains both model predictions


In [ ]:
def assert_monotonic_dates(df, name='df'):
    if not df['date'].is_monotonic_increasing:
        raise ValueError(f'{name} dates are not sorted ascending')

assert_monotonic_dates(arima_fc, 'arima_fc')
assert_monotonic_dates(lstm_fc, 'lstm_fc')
assert_monotonic_dates(merged_fc, 'merged_fc')

dates_arima = set(arima_fc['date'])
dates_merged = set(merged_fc['date'])

print('Dates in ARIMA but not merged:', len(dates_arima - dates_merged))
print('Dates in merged but not ARIMA:', len(dates_merged - dates_arima))

missing_cols = [c for c in ['y_true', 'arima_pred', 'lstm_pred'] if c not in merged_fc.columns]
if missing_cols:
    raise ValueError(f'Merged forecasts missing columns: {missing_cols}')

print('Merged columns OK.')


## 3) Load model specifications (ARIMA/SARIMA parameters + LSTM architecture)

These JSON files are the **documentation artifacts** required for the report.


In [ ]:
with open(ARIMA_PARAMS_PATH, 'r', encoding='utf-8') as f:
    arima_params = json.load(f)

with open(LSTM_ARCH_PATH, 'r', encoding='utf-8') as f:
    lstm_info = json.load(f)

display(arima_params)
display(lstm_info)


### ARIMA/SARIMA summary
Key items to report:
- `order = (p,d,q)`
- `seasonal_order = (P,D,Q,m)` if seasonal
- selection criterion (AIC)


In [ ]:
arima_summary = {
    'asset': arima_params.get('asset'),
    'target_col': arima_params.get('target_col'),
    'seasonal': arima_params.get('seasonal'),
    'm': arima_params.get('m'),
    'order': arima_params.get('order'),
    'seasonal_order': arima_params.get('seasonal_order'),
    'aic': arima_params.get('aic'),
    'bic': arima_params.get('bic'),
}

display(pd.DataFrame([arima_summary]))


### LSTM architecture & training summary
We report:
- lookback window
- features used
- scaling strategy
- layers/units/dropout
- epochs/batch size/learning rate


In [ ]:
run_cfg = lstm_info.get('run', {})
lstm_summary = {
    'lookback': run_cfg.get('lookback'),
    'horizon': run_cfg.get('horizon'),
    'n_features': len(run_cfg.get('feature_cols', [])),
    'feature_cols': ', '.join(run_cfg.get('feature_cols', [])),
    'scaler_type': run_cfg.get('scaler_type'),
    'units_1': run_cfg.get('units_1'),
    'units_2': run_cfg.get('units_2'),
    'dropout': run_cfg.get('dropout'),
    'rec_dropout': run_cfg.get('rec_dropout'),
    'epochs': run_cfg.get('epochs'),
    'batch_size': run_cfg.get('batch_size'),
    'learning_rate': run_cfg.get('learning_rate'),
}

display(pd.DataFrame([lstm_summary]).T.rename(columns={0:'value'}))


## (Demo) Build an ARIMA model on returns (stationary data) and forecast prices

This section demonstrates the classical workflow: model *returns* (approximately stationary) and map back to prices.

> This is **illustrative** only. The **official** ARIMA/SARIMA results used for scoring are the script-generated artifacts loaded above.


In [ ]:
if 'tsla_full' in globals():
    import statsmodels.api as sm

    demo_train_end = pd.to_datetime(split_info['cutoff_date'])
    demo_train = tsla_full[tsla_full['date'] <= demo_train_end].copy()
    demo_test  = tsla_full[tsla_full['date'] >  demo_train_end].copy()

    # Stationary target: log returns
    r_train = demo_train['log_ret'].dropna()

    # Small ARIMA order as a demonstration
    demo_order = (1, 0, 1)
    model = sm.tsa.ARIMA(r_train, order=demo_order)
    res = model.fit()

    steps = len(demo_test)
    r_fc = res.forecast(steps=steps)
    r_fc.index = demo_test['date'].values

    last_train_price = float(demo_train['adj_close'].iloc[-1])
    price_fc = last_train_price * np.exp(np.cumsum(r_fc.values))

    demo_arima_df = pd.DataFrame({
        'date': demo_test['date'].values,
        'actual_price': demo_test['adj_close'].values,
        'demo_arima_price_fc': price_fc
    })

    display(demo_arima_df.head())

    plt.figure(figsize=(12,4))
    plt.plot(demo_arima_df['date'], demo_arima_df['actual_price'], label='Actual', linewidth=2)
    plt.plot(demo_arima_df['date'], demo_arima_df['demo_arima_price_fc'], label=f'Demo ARIMA on returns {demo_order}', linewidth=1.5)
    plt.title('Demo: ARIMA on log returns → implied price forecast (test period)')
    plt.xlabel('Date'); plt.ylabel('Price')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print('Skipping ARIMA-on-returns demo: underlying series not loaded.')


## (Demo) Prepare data for LSTM (scaling + sequence creation)

This section demonstrates the LSTM-specific prep steps. The actual model training is done in scripts.


In [ ]:
if FEATURES_TRAIN_PATH.exists() and FEATURES_TEST_PATH.exists():
    feat_train = pd.read_parquet(FEATURES_TRAIN_PATH).copy()
    feat_test  = pd.read_parquet(FEATURES_TEST_PATH).copy()

    for df in (feat_train, feat_test):
        df['date'] = pd.to_datetime(df['date'])
        df.sort_values('date', inplace=True)

    feature_cols = run_cfg.get('feature_cols', [])
    lookback = int(run_cfg.get('lookback', 30))

    X_train_df = feat_train[feature_cols].copy()
    X_test_df  = feat_test[feature_cols].copy()

    from sklearn.preprocessing import MinMaxScaler
    scaler_X = MinMaxScaler()
    X_train_scaled = scaler_X.fit_transform(X_train_df.values)
    X_test_scaled  = scaler_X.transform(X_test_df.values)

    def make_sequences(X2d, lookback):
        X_seq = []
        for i in range(lookback, len(X2d)):
            X_seq.append(X2d[i - lookback:i, :])
        return np.array(X_seq)

    X_train_seq = make_sequences(X_train_scaled, lookback)
    X_test_seq  = make_sequences(X_test_scaled, lookback)

    print('Feature columns:', feature_cols)
    print('lookback:', lookback)
    print('X_train_scaled:', X_train_scaled.shape)
    print('X_train_seq:   ', X_train_seq.shape, '(samples, timesteps, features)')
    print('X_test_seq:    ', X_test_seq.shape)

    if 'adj_close' in feature_cols:
        adj_idx = feature_cols.index('adj_close')
        y_train_demo = X_train_scaled[lookback:, adj_idx]
        y_test_demo  = X_test_scaled[lookback:, adj_idx]
        print('y_train_demo:', y_train_demo.shape)
        print('y_test_demo: ', y_test_demo.shape)
    else:
        print('Note: adj_close not found in feature_cols; check lstm_architecture.json')
else:
    print('Skipping LSTM prep demo: feature parquet files not found.')


## 4) Required metrics table (MAE / RMSE / MAPE)

This is the required deliverable: **`model_comparison.csv`**.


In [ ]:
comparison = pd.read_csv(MODEL_COMPARISON_PATH)
display(comparison)


### Sanity check: recompute metrics from merged forecasts
We recompute MAE/RMSE/MAPE directly from `tsla_forecasts_merged.csv` to confirm consistency.


In [ ]:
def mae(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.mean(np.abs(y_true - y_pred)))

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def mape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.maximum(np.abs(y_true), eps)
    return float(np.mean(np.abs((y_true - y_pred) / denom)) * 100.0)

y_true = merged_fc['y_true'].values
arima_pred = merged_fc['arima_pred'].values
lstm_pred = merged_fc['lstm_pred'].values

recalc = pd.DataFrame([
    {'model': 'ARIMA_recalc', 'MAE': mae(y_true, arima_pred), 'RMSE': rmse(y_true, arima_pred), 'MAPE_pct': mape(y_true, arima_pred)},
    {'model': 'LSTM_recalc',  'MAE': mae(y_true, lstm_pred),  'RMSE': rmse(y_true, lstm_pred),  'MAPE_pct': mape(y_true, lstm_pred)},
])

display(recalc)


## 5) Visual comparison: Actual vs Forecasts on test period

We display the saved figure if it exists; otherwise we generate the plot inline.


In [ ]:
if os.path.exists(FORECAST_PLOT_PATH):
    from PIL import Image
    img = Image.open(FORECAST_PLOT_PATH)
    display(img)
else:
    print('Saved plot not found; generating inline plot...')
    df = merged_fc.sort_values('date')

    plt.figure(figsize=(12, 5))
    plt.plot(df['date'], df['y_true'], label='Actual (TSLA adj_close)', linewidth=2)
    plt.plot(df['date'], df['arima_pred'], label='ARIMA', linewidth=1.5)
    plt.plot(df['date'], df['lstm_pred'], label='LSTM (multivariate)', linewidth=1.5)
    plt.title('TSLA Forecasts on Test Period')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()


## 6) Extra section — deliverable proof (head/tail) + error distribution diagnostics

This section strengthens the report by:
- Showing first/last rows of forecast artifacts (alignment proof)
- Providing error distribution statistics (bias, dispersion, quantiles)
- Showing how often one model beats the other per-day (absolute error)


In [ ]:
def preview_df(df, name, n=5):
    print(f'\n{name} — head({n})')
    display(df.head(n))
    print(f'\n{name} — tail({n})')
    display(df.tail(n))

preview_df(arima_fc, 'ARIMA forecast CSV')
preview_df(lstm_fc, 'LSTM forecast CSV')
preview_df(merged_fc, 'Merged forecasts CSV')


### 6.2 Additional error diagnostics (per model) + win-rate

- loads the saved diagnostics (error_diagnostics.csv) as the report source-of-truth
- recomputes win-rate from merged_fc (optional) and checks if it matches the artifact


In [ ]:
diag_script = pd.read_csv(ERROR_DIAGNOSTICS_PATH)

diag_models = diag_script[diag_script['model'].isin(['ARIMA', 'LSTM_multivariate'])].copy()
display(diag_models)

df = merged_fc.sort_values('date').copy()
comp = df[['date', 'y_true', 'arima_pred', 'lstm_pred']].dropna().copy()

comp['abs_err_arima'] = (comp['arima_pred'] - comp['y_true']).abs()
comp['abs_err_lstm']  = (comp['lstm_pred']  - comp['y_true']).abs()

n = len(comp)
lstm_win_rate_pct = (comp['abs_err_lstm'] < comp['abs_err_arima']).mean() * 100
arima_win_rate_pct = (comp['abs_err_arima'] < comp['abs_err_lstm']).mean() * 100
tie_rate_pct = (comp['abs_err_arima'] == comp['abs_err_lstm']).mean() * 100

win_check = pd.DataFrame(
    {
        'win_rate_vs_other_pct_recalc': [arima_win_rate_pct, lstm_win_rate_pct, tie_rate_pct],
        'n_compared': [n, n, n],
    },
    index=['ARIMA', 'LSTM_multivariate', 'TIES']
)

display(win_check)

merged_check = diag_script[['model', 'win_rate_vs_other_pct']].merge(
    win_check.reset_index().rename(columns={'index': 'model'}),
    on='model',
    how='inner'
)

merged_check['abs_diff_pct_points'] = (
    merged_check['win_rate_vs_other_pct'] - merged_check['win_rate_vs_other_pct_recalc']
).abs()

display(merged_check)

max_diff = merged_check['abs_diff_pct_points'].max()
print(f'Max absolute difference (pct points) between artifact and notebook recalc: {max_diff:.6f}')


### 6.4 Error over time (optional diagnostic plot)
This plot helps identify whether one model drifts or fails during volatility regimes.


In [ ]:
plot_df = merged_fc.sort_values('date').copy()

if 'err_arima' not in plot_df.columns or 'err_lstm' not in plot_df.columns:
    y_col = 'y_true' if 'y_true' in plot_df.columns else ('actual' if 'actual' in plot_df.columns else None)
    arima_col = 'arima_pred' if 'arima_pred' in plot_df.columns else ('arima' if 'arima' in plot_df.columns else None)
    lstm_col = 'lstm_pred' if 'lstm_pred' in plot_df.columns else ('lstm' if 'lstm' in plot_df.columns else None)
    if not (y_col and arima_col and lstm_col):
        raise KeyError(
            'Expected columns for error plotting not found. '
            f'Need y_true/actual, arima_pred/arima, lstm_pred/lstm. Got: {list(plot_df.columns)}'
        )
    plot_df['err_arima'] = plot_df[arima_col] - plot_df[y_col]
    plot_df['err_lstm'] = plot_df[lstm_col] - plot_df[y_col]

plt.figure(figsize=(12, 4))
plt.plot(plot_df['date'], plot_df['err_arima'], label='ARIMA error (pred-actual)', linewidth=1)
plt.plot(plot_df['date'], plot_df['err_lstm'], label='LSTM error (pred-actual)', linewidth=1)
plt.axhline(0, color='black', linewidth=1, alpha=0.7)
plt.title('Forecast Errors Over Test Period')
plt.xlabel('Date')
plt.ylabel('Error')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


## 7) Discussion (brief, metrics-grounded)

### Model behavior considerations
- **ARIMA/SARIMA** (univariate) models linear autocorrelation structure in `adj_close`. It tends to perform well when the series is relatively smooth and the next-step value is strongly related to recent values.
- **Multivariate LSTM** can exploit nonlinear relationships and additional predictors (OHLCV and engineered indicators), but it is sensitive to feature scaling, lookback choice, and regime shifts/volatility spikes.

### Which model is better here?
We choose the model with **lower RMSE** (primary), then MAE and MAPE as supporting metrics.

### Notes on validity / leakage
- Chronological split is enforced by cutoff date.
- LSTM scalers are fit on **train only** (per scripts) to avoid leakage.


In [ ]:
# Auto-generate a short conclusion snippet based on RMSE
cmp = comparison.copy().sort_values('RMSE')
best = cmp.iloc[0].to_dict()
runner_up = cmp.iloc[1].to_dict() if len(cmp) > 1 else None

print('Best model by RMSE:')
print(best)

if runner_up:
    print('\nRunner-up:')
    print(runner_up)

print('\nConclusion draft:')
print(
    f"Based on the test-period evaluation, the best-performing model is {best['model']} "
    f"with RMSE={best['RMSE']:.4f}, MAE={best['MAE']:.4f}, MAPE={best['MAPE_pct']:.2f}%. "
    'This indicates it better captures the short-horizon dynamics of TSLA adj_close over the held-out period.'
)


## 8) Deliverables checklist (for submission)

Confirm these files exist in your repo after running scripts:

- **Splits & features**
  - `data/task2/splits/tsla_train.parquet`
  - `data/task2/splits/tsla_test.parquet`
  - `data/task2/features/tsla_features_train.parquet`
  - `data/task2/features/tsla_features_test.parquet`

- **Model artifacts**
  - `outputs/task2/models/lstm_model.keras`
  - (optional) `outputs/task2/models/arima_model.pkl`

- **Forecasts (aligned to test dates)**
  - `outputs/task2/forecasts/tsla_arima_forecast.csv`
  - `outputs/task2/forecasts/tsla_lstm_forecast.csv`
  - `outputs/task2/forecasts/tsla_forecasts_merged.csv`

- **Metrics & documentation**
  - `outputs/task2/metrics/model_comparison.csv`  ✅ required table
  - `outputs/task2/metrics/arima_params.json`      ✅ ARIMA/SARIMA parameters
  - `outputs/task2/metrics/lstm_architecture.json` ✅ LSTM architecture
  - `outputs/task2/metrics/split_info.json`        ✅ cutoff proof

- **Figures**
  - `outputs/task2/figures/forecast_test_period.png`
